In [2]:
import pandas as pd

dataset='coco'
test_ratio=[0.8,0.5,0.2]
refs=""
contents=[]

for i in range(len(test_ratio)):
    prompt_path=pd.read_parquet('blip2_ft_'+dataset+'/result/blip2_'+str(test_ratio[i])+'_10_test.parquet')
    contents.append(prompt_path['generated_caption'].tolist())
refs=prompt_path['reference_prompt'].tolist()

In [3]:
path='blip2_ft_'+dataset+'/result/blip2_0.0_10_test.txt'
with open(path, 'r', encoding='utf-8') as file:
    single=file.readlines()
contents.append(single)
len(contents)

4

In [4]:
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

scores = []

for i in tqdm(range(len(contents[0]))):
    score = 0.0
    base_emd = model.encode(refs[i])
    for j in range(len(contents)):
        each_emd = model.encode(contents[j][i])
        cosine_sim = util.pytorch_cos_sim(base_emd, each_emd)
        score += cosine_sim.item()
    scores.append(score)

/home/fenghe/anaconda3/envs/ldm/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/fenghe/anaconda3/envs/ldm/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
100%|██████████| 688/688 [00:13<00:00, 50.83it/s]


In [5]:
import numpy as np

scores = np.array(scores)
k = 100

topk_indices = scores.argsort()[-k:][::-1]
topk_values = scores[topk_indices]

print("Top-k 索引:", topk_indices.tolist())
print("Top-k 值:", topk_values.tolist())

Top-k 索引: [653, 364, 307, 207, 48, 407, 229, 358, 384, 26, 243, 122, 513, 11, 570, 605, 367, 84, 153, 18, 33, 564, 460, 19, 287, 498, 457, 29, 511, 266, 283, 508, 554, 284, 91, 175, 251, 355, 36, 158, 215, 217, 456, 383, 434, 349, 256, 572, 538, 478, 293, 427, 587, 370, 489, 392, 388, 421, 481, 17, 606, 246, 483, 285, 97, 404, 584, 412, 539, 597, 172, 515, 557, 558, 31, 524, 639, 419, 166, 615, 322, 560, 306, 366, 318, 98, 154, 54, 679, 155, 684, 578, 101, 173, 121, 312, 439, 609, 94, 49]
Top-k 值: [3.9977479577064514, 3.9724597930908203, 3.94272243976593, 3.905319929122925, 3.8981293439865112, 3.895027816295624, 3.8787229657173157, 3.8762205839157104, 3.8689703941345215, 3.868078589439392, 3.8634254932403564, 3.857824742794037, 3.8435518741607666, 3.834706425666809, 3.8291919231414795, 3.8245806097984314, 3.807273209095001, 3.8044159412384033, 3.8038649559020996, 3.8026227951049805, 3.8021270632743835, 3.7951820492744446, 3.792963147163391, 3.7926183938980103, 3.7908726930618286, 3.780

In [7]:
for idx in topk_indices:
    print(refs[idx])
    for i in range(len(contents)):
        print(idx,contents[i][idx])

A motorcycle parked in front of a graffiti covered building.
653 A motorcycle parked in front of a graffiti covered building
653 A motorcycle parked in front of a graffiti covered building.
653 A motorcycle parked in front of a graffiti covered building.
653 A motorcycle parked in front of a graffiti covered building.

A group of people preparing food in a kitchen.
364 A group of people in a kitchen preparing food
364 A group of people in a kitchen preparing food
364 A group of people in a kitchen preparing food.
364 A group of people preparing food in a kitchen.

A large airplane is on the airport runway.
307 A large airplane on the runway at an airport.
307 A large airplane is on the runway at the airport.
307 A large airplane is on the runway at the airport.
307 A large airplane is on the airport runway.

A cat lying down on the road next to a car.
207 A cat laying on the road next to a car
207 A cat laying on the road next to a car.
207 A cat laying on the road next to a car.
207 A